### Excercise to Understand Basic data Processing Using Spark and getting readyfor using Spark for Machine learning.

In [1]:
# 0.0 Calling Requried Libraries to make things smoother
from IPython.core.interactiveshell import InteractiveShell # to make better interaction with shell
from IPython.core.display import display, HTML # to display output as html to avoid truncation or mis reading
from pyspark.sql.functions import col,sum # col() returns a Column (values) based on a given column name. sum will be used as normal sum function.
from pyspark.sql.types import DoubleType #  Import class 'DoubleType' that represents double data type.
from pyspark.ml.feature import StringIndexer # For Processing Categorical String Variables
from pyspark.ml.feature import OneHotEncoderEstimator # for preparing Categorical String Indexer output columns for Machine learning and analysis
from pyspark.ml import Pipeline # to create Pipeline to perform multiple Stages in one go.
from pyspark.ml.feature import VectorAssembler # to create One feature columns assembled by adding the Multiple columns of dataframe.

#### Formatting The Spark and Verifying Spark instances

In [2]:
#1.1 Display multiple outputs from a cell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#1.2 Increase cell width to display wide columnar output
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# 1.3 To check if spark context is ,sc, and spark session is, spark: 
#        try the following two commands. Also right-click on Spark_UI hyperlink
#        to open another site.
#      Spark 'driver' process manifests itself through 'spark' session object

sc
spark

<SparkContext master=local[*] appName=PySparkShell>

#### Data was moved to Hadoop File system using below commands ( Assignemnt Q.1 )
        ''' 
        cd ~
         hdfs dfs -put /home/ashok/Excercise/bank.csv hdfs://localhost:9000/user/ashok/data_files/bank.csv
         hdfs dfs -ls -h hdfs://localhost:9000/user/ashok/data_files
        '''
        

#### Reading Dataset from Hadoop file system. ( Assignment Q.2)

In [5]:
# 2. Read, transform and understand the data
#      https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.csv
#    pyspark creates a spark-session variable: spark
df = spark.read.csv(
                   path = "hdfs://localhost:9000/user/ashok/data_files/bank.csv",   # path to hadoop
                   header = True,
                   inferSchema= True,           # Infer datatypes automatically
                   sep = "," ,                  # Can be any character (check \t)
                   )

#### Creating Cache of the dataframe and Looking at verious details of Data frame "df" ( Assignment Q.3)

In [6]:
# 3.1 We also cache the data so that we only read it from disk once.
df.cache()
df.is_cached            # Checks if df is cached

DataFrame[age: int, job: string, marital: string, education: string, default: string, balance: int, housing: string, loan: string, contact: string, day: int, month: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, deposit: string]

True

In [7]:
# 3.2 Size of data df
df.count(), len(df.columns)
# Number of Rows
df.count()
# Number of Columns
len(df.columns)

(11162, 17)

11162

17

In [8]:
# 3.3 Show data. Note that 
df.show(5)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|  services|married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|    admin.|married| tertiary|     no|    184|     no|  no|unknown|  5| 

In [9]:
# 3.4 Have a look at the dataframe schema,i.e. the structure of the DataFrame
   
df.printSchema()
df.dtypes

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'int'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('deposit', 'string')]

In [10]:
# 3.5 Check missing values:
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()

+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|  0|  0|      0|        0|      0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|      0|
+---+---+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+



#### Processing the data in "df" to work on required Data, dropping unnecessary columns. (Assignment Q.4)

In [11]:
# 4.0 To select Data only from required columns i.e.:['age', 'job', 'marital', 'education', 'default',  'balance','housing', 'loan', 'contact',
# 'duration',  'campaign', 'pdays', 'previous', 'poutcome', 'deposit']
# drop individual columns to get required columns. as per assignment.

df = df.drop("day").drop("month")
df.show(3)

+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
|age|       job|marital|education|default|balance|housing|loan|contact|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|    1042|       1|   -1|       0| unknown|    yes|
| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|    1467|       1|   -1|       0| unknown|    yes|
| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|    1389|       1|   -1|       0| unknown|    yes|
+---+----------+-------+---------+-------+-------+-------+----+-------+--------+--------+-----+--------+--------+-------+
only showing top 3 rows



#### Processing String Categorical Columns using StringIndexer and OneHotEncoderEstimator. Then creating Stages for using in Pipeline (Assignment Q.5)

In [12]:
# v) The list of categorical columns is as follows:

catCols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']

# Create a pipeline to transform each one of the categorical columns to as many Onehotencoded columns	
# by first using  StringIndexer and then OneHotEncoder. 

# 5 ======= build stages ================
# Creating StringIndexer Stage for all the 
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='stringindexed_' + c) for c in catCols]
# Creating StringIndexer Stage for all the
in_cols = ['stringindexed_' + c for c in catCols]
out_cols = ['onehotencoded_' + c  for c in catCols]
onehotencoder_stages = [OneHotEncoderEstimator(inputCols=in_cols, outputCols=out_cols)]

#  Create Pipeline 
pipeline = Pipeline(stages=stringindexer_stages+onehotencoder_stages)

##  Fit pipeline model
pipeline_mode = pipeline.fit(df)

## Transform data
df_coded = pipeline_mode.transform(df)
df_coded.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'deposit',
 'stringindexed_job',
 'stringindexed_marital',
 'stringindexed_education',
 'stringindexed_default',
 'stringindexed_housing',
 'stringindexed_loan',
 'stringindexed_contact',
 'stringindexed_poutcome',
 'onehotencoded_default',
 'onehotencoded_contact',
 'onehotencoded_education',
 'onehotencoded_housing',
 'onehotencoded_marital',
 'onehotencoded_loan',
 'onehotencoded_poutcome',
 'onehotencoded_job']

#### Using Vector Assembler to use OneHotEncoded Columns and Numeric Columns for Assembling featured columns. ( Assignment Q.6)

In [13]:
# vi) Use VectorAssembler to aseemble all the OneHotEncoded columns and the following numerical columns in one column.
# Call this new assembled column as: 'rawFeatures' :

numericCols = ['age', 'balance', 'duration',  'campaign', 'pdays', 'previous']

# 6.1 We will now cast the rest of  columns into doubletype in one go as VectorAssembler will need that kind of data for Processing and making it fearured column
for l in numericCols:
    df_coded = df_coded.withColumn(l, col(l).cast(DoubleType()))
    
df_coded.dtypes

[('age', 'double'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'double'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('duration', 'double'),
 ('campaign', 'double'),
 ('pdays', 'double'),
 ('previous', 'double'),
 ('poutcome', 'string'),
 ('deposit', 'string'),
 ('stringindexed_job', 'double'),
 ('stringindexed_marital', 'double'),
 ('stringindexed_education', 'double'),
 ('stringindexed_default', 'double'),
 ('stringindexed_housing', 'double'),
 ('stringindexed_loan', 'double'),
 ('stringindexed_contact', 'double'),
 ('stringindexed_poutcome', 'double'),
 ('onehotencoded_default', 'vector'),
 ('onehotencoded_contact', 'vector'),
 ('onehotencoded_education', 'vector'),
 ('onehotencoded_housing', 'vector'),
 ('onehotencoded_marital', 'vector'),
 ('onehotencoded_loan', 'vector'),
 ('onehotencoded_poutcome', 'vector'),
 ('onehotencoded_job', 'vector')]

In [14]:
# 6.2 Which columns to concatenate: as per Question in Assignment : the OneHotEncoded columns and the following numerical columns in one column.
Feature_columns= numericCols+out_cols
# Now we will see clumn names, that will be helpful as featuresd single Indexed column.
Feature_columns

['age',
 'balance',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'onehotencoded_job',
 'onehotencoded_marital',
 'onehotencoded_education',
 'onehotencoded_default',
 'onehotencoded_housing',
 'onehotencoded_loan',
 'onehotencoded_contact',
 'onehotencoded_poutcome']

In [15]:
# 6.3   Create an instance of VectorAssembler class.
#          This object will be used to transfrom data farme,
#           as: vectorassembler.fit(df)
vectorassembler = VectorAssembler(
                                  inputCols=Feature_columns,
                                  outputCol="rawFeatures"
                                 )

#### Printing Output.rawFeatures Column to verify what all are included in raw features column. (Assignment Q.7)

In [16]:
# Now Assembling columns in Vector Assembler on NUmeric Col and Onehotencoded columns
output = vectorassembler.transform(df_coded)
# Just see the 'rawFeatures' column
output.select('rawFeatures').show(3,truncate=False)

+---------------------------------------------------------------------------------------------------+
|rawFeatures                                                                                        |
+---------------------------------------------------------------------------------------------------+
|(30,[0,1,2,3,4,9,17,19,22,24,26,27],[59.0,2343.0,1042.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])     |
|(30,[0,1,2,3,4,9,17,19,22,23,24,26,27],[56.0,45.0,1467.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(30,[0,1,2,3,4,8,17,19,22,24,26,27],[41.0,1270.0,1389.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])     |
+---------------------------------------------------------------------------------------------------+
only showing top 3 rows

